In [1]:
import time
import osmnx as ox
import pandas as pd
import geopandas as gpd

In [2]:
def get_poi_query(north, south, east, west, key = None,values = None, timeout=180, maxsize='',
                    custom_settings=None):
    '''
    仿写osmnx
    根据输入key、values、四边界，返回OverpassAPI的query字符串
    '''
    # use custom settings if delivered, otherwise just the default ones.
    if custom_settings:
        overpass_settings = custom_settings
    else:
        overpass_settings = ox.settings.default_overpass_query_settings.format(timeout=timeout, maxsize=maxsize)

    if key and values:
        # Overpass QL template
        query_template = ('{settings};((node["{key}"~"{values}"]({south:.6f},'
                          '{west:.6f},{north:.6f},{east:.6f});(._;>;););(way["{key}"~"{values}"]({south:.6f},'
                          '{west:.6f},{north:.6f},{east:.6f});(._;>;););(relation["{key}"~"{values}"]'
                          '({south:.6f},{west:.6f},{north:.6f},{east:.6f});(._;>;);););out;')

        # Parse amenties
        query_str = query_template.format(key=key,values="|".join(values), north=north, south=south, east=east, west=west,timeout=timeout, maxsize=maxsize, settings=overpass_settings)
    else:
        # Overpass QL template
        query_template = ('{settings};((node["amenity"]({south:.6f},'
                          '{west:.6f},{north:.6f},{east:.6f});(._;>;););(way["amenity"]({south:.6f},'
                          '{west:.6f},{north:.6f},{east:.6f});(._;>;););(relation["amenity"]'
                          '({south:.6f},{west:.6f},{north:.6f},{east:.6f});(._;>;);););out;')

        # Parse amenties
        query_str = query_template.format(north=north, south=south, east=east, west=west,
                                          timeout=timeout, maxsize=maxsize, settings=overpass_settings)
    return query_str

In [3]:
def create_POI_gdf(north, south, east, west, key = None,values = None,custom_settings=None,timeout=180):
    '''
    仿写osmnx
    从OverpassAPI获取response，并解析nodes ways relations，返回全部POI的GeoDataFrame
    '''
    query = get_poi_query(north=north,south=south,east=east,west=west,key=key,values=values,
                                custom_settings=custom_settings)
#     print(query)
    responses = ox.overpass_request(data={'data': query}, timeout=timeout)
    # Parse coordinates from all the nodes in the response
    coords = ox.parse_nodes_coords(responses)

    # POI nodes
    poi_nodes = {}

    # POI ways
    poi_ways = {}

    # A list of POI relations
    relations = []

    for result in responses['elements']:
        if result['type'] == 'node' and 'tags' in result:
            poi = ox.parse_osm_node(response=result)
            # Add element_type
            poi['element_type'] = 'node'
            # Add to 'pois'
            poi_nodes[result['id']] = poi
        elif result['type'] == 'way':
            # Parse POI area Polygon
            poi_area = ox.parse_polygonal_poi(coords=coords, response=result)
            if poi_area:
                # Add element_type
                poi_area['element_type'] = 'way'
                # Add to 'poi_ways'
                poi_ways[result['id']] = poi_area

        elif result['type'] == 'relation':
            # Add relation to a relation list (needs to be parsed after all nodes and ways have been parsed)
            relations.append(result)

    # Create GeoDataFrames
    gdf_nodes = gpd.GeoDataFrame(poi_nodes).T
    gdf_nodes.crs = ox.settings.default_crs

    gdf_ways = gpd.GeoDataFrame(poi_ways).T
    gdf_ways.crs = ox.settings.default_crs

    # Parse relations (MultiPolygons) from 'ways'
    gdf_ways = ox.parse_osm_relations(relations=relations, osm_way_df=gdf_ways)

    # Combine GeoDataFrames
    gdf = gdf_nodes.append(gdf_ways, sort=False)

    return gdf

In [4]:
def get_retail_format_data(INPUT_CSV):
    '''
    输入校园POI的csv文件，依次获取餐饮、零售、生活服务、休闲娱乐4类业态数量和各自占比，返回DataFrame
    '''
    starttime = time.time()
    print(starttime)
    catering = {'key':'amenity', 'values':['restaurant','fast_food','bbq','bar','pub','food_court']}
    shopping = {'key':'shop', 'values':['convenience','supermarket','dairy','bakery','confectionery','coffee','chocolate','ice_cream',
                                        'greengrocer','craft','stationery','book','newsagent']}
    service1 = {'key':'shop', 'values':['hairdresser','copyshop','laundry','dry_cleaning','optician','bicycle']}
    service2 = {'key':'amenity', 'values':['bicycle_repair_station','bicycle_rental','clinic','dentist','pharmacy','hospital']}
    leisure1 = {'key':'amenity', 'values':['library','theatre','cinema','arts_centre','internet_cafe']}
    leisure2 = {'key':'leisure', 'values':['fitness_centre','fitness_station']}
    leisure3 = {'key':'tourism', 'values':['museum','gallery']}
    
    campus = pd.read_csv(INPUT_CSV,encoding='UTF-8')
    data_stack = []
    for i in range(len(campus)):
        osmid = campus.iloc[i]['OSMID']
        length_lat = campus.iloc[i]['north']-campus.iloc[i]['south']
        width_lon = campus.iloc[i]['east']-campus.iloc[i]['west']
        bbox_north = campus.iloc[i]['north']+length_lat
        bbox_south = campus.iloc[i]['south']-length_lat
        bbox_east = campus.iloc[i]['east']+width_lon
        bbox_west = campus.iloc[i]['west']-width_lon
        
        catering_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=catering['key'],values=catering['values'])
        shopping_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=shopping['key'],values=shopping['values'])
        service1_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=service1['key'],values=service1['values'])
        service2_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=service2['key'],values=service2['values'])
        leisure1_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=leisure1['key'],values=leisure1['values'])
        leisure2_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=leisure2['key'],values=leisure2['values'])
        leisure3_gdf = create_POI_gdf(north=bbox_north,south=bbox_south,east=bbox_east,west=bbox_west,key=leisure3['key'],values=leisure3['values'])
        
        num_catering = len(catering_gdf)
        num_shopping = len(shopping_gdf)
        num_service = len(service1_gdf)+len(service2_gdf)
        num_leisure = len(leisure1_gdf)+len(leisure2_gdf)+len(leisure3_gdf)
        num_sum = num_catering + num_shopping + num_service + num_leisure
        if num_sum != 0:
            data_stack.append([osmid,num_sum,num_catering,num_shopping,num_service,num_leisure,
                        num_catering/num_sum,num_shopping/num_sum,num_service/num_sum,num_leisure/num_sum])
        else:
            data_stack.append([osmid,num_sum,num_catering,num_shopping,num_service,num_leisure,0,0,0,0])
        if (i+1)%50==0:
            print(i+1,' data parsed')
    endtime = time.time()
    print('step total time:', round(endtime - starttime, 2),'secs')
    print('retail format data parsed!')
    return pd.DataFrame(data_stack,columns=['OSMID','sum','catering','shopping','service','leisure','ratio_catering','ratio_shopping','ratio_service','ratio_leisure'])

In [13]:
INPUT_CSV = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_campus/russia_latest_university.csv'
retail_format_data = get_retail_format_data(INPUT_CSV)
retail_format_data

1590116511.6405458
1  data parsed
51  data parsed
101  data parsed
151  data parsed
201  data parsed
251  data parsed
301  data parsed
351  data parsed
401  data parsed
451  data parsed
501  data parsed
551  data parsed
601  data parsed
651  data parsed
701  data parsed
751  data parsed
801  data parsed
851  data parsed
901  data parsed
step total time: 34859.04 secs
retail format data parsed!


,OSMID,sum,catering,shopping,service,leisure,ratio_catering,ratio_shopping,ratio_service,ratio_leisure
0,23572862,9,1,2,6,0,0.111111,0.222222,0.666667,0.00000
1,23573799,8,3,1,2,2,0.375000,0.125000,0.250000,0.25000
2,25117044,32,13,4,12,3,0.406250,0.125000,0.375000,0.09375
3,26174972,0,0,0,0,0,0.000000,0.000000,0.000000,0.00000
4,26940718,4,1,1,1,1,0.250000,0.250000,0.250000,0.25000
...,...,...,...,...,...,...,...,...,...,...
907,775252145,6,0,3,3,0,0.000000,0.500000,0.500000,0.00000
908,777913082,2,0,1,1,0,0.000000,0.500000,0.500000,0.00000
909,780566770,1,0,1,0,0,0.000000,1.000000,0.000000,0.00000
910,782726493,15,6,5,4,0,0.400000,0.333333,0.266667,0.00000


In [14]:
OUTPUT_CSV = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/retail_format_data/russia_retail_format.csv'
retail_format_data.to_csv(OUTPUT_CSV,index=0,encoding='utf-8-sig')